In [1]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sgp4.ext import invjday
from sgp4.api import Satrec, jday
from joblib import Parallel, delayed
from datetime import datetime, timedelta
from skyfield.api import load, wgs84, EarthSatellite

In [2]:
username = 'asas4539@hanyang.ac.kr'
password = 'onsaemiro1729!!'

In [3]:
def get_tle(number):
    # Space-Track API의 URL
    url = "https://www.space-track.org/ajaxauth/login"

    # 요청에 필요한 인증 정보
    payload = {"identity": username, "password": password}

    # Session 객체 생성
    session = requests.Session()

    # 로그인 요청
    response = session.post(url, data=payload)

    if response.status_code != 200:
        print("로그인에 실패했습니다.")
        return None

    # TLE 요청
    response = session.get(
        f"https://www.space-track.org/basicspacedata/query/class/tle/NORAD_CAT_ID/{number}/EPOCH/2022-12-31--2024-07-31/orderby/EPOCH%20asc/format/3le/emptyresult/show"
    )

    return response

In [4]:
def set_plot_params(title, ylabel):
    plt.title(title)
    plt.ylabel(ylabel, fontsize=12)
    plt.xlabel("Date", fontsize=12)
    plt.yticks(fontsize=12)
    plt.xticks(rotation=30, fontsize=12)
    plt.gca().xaxis.set_major_locator(plt.MultipleLocator(30))


def calculate_ylim(data):
    ylim_min = min(data) - (max(data) - min(data)) * 0.1
    ylim_max = max(data) + (max(data) - min(data)) * 0.1
    return ylim_min, ylim_max


def draw_graph(data: pd.DataFrame, title: str, ylabel: str, date_lst: list, color="blue"):
    '''
    :param data: 시간 인덱스와 그래프로 표시할 정보가 있는 데이터프레임
    :param title: 그래프 제목
    :param ylabel: 그래프 y축 단위
    :param date_lst: 아웃라이어 날짜인덱스 리스트
    :param color: 그래프 색깔
    :return: 
    '''
    plt.figure(figsize=(10, 5))
    plt.plot(data.index, data.values, color=color)
    for date in date_lst:
        plt.axvline(pd.to_datetime(date), color='red', linestyle='--', label='Highlight Date')
    set_plot_params(title, ylabel)
    ylim_min, ylim_max = calculate_ylim(list(data.values))
    plt.ylim(ylim_min, ylim_max)
    plt.show()

In [5]:
def get_info(seg):
    change_df_seg = seg

    start_info = change_df_seg.iloc[0]
    epoch = datetime(start_info['year'], start_info['month'], start_info['day'], start_info['hour'],
                     start_info['minute'], int(start_info['second']))
    epoch += timedelta(minutes=1)

    end_info = change_df_seg.iloc[-1]
    end_epoch = datetime(end_info['year'], end_info['month'], end_info['day'], end_info['hour'], end_info['minute'],
                         int(end_info['second']))

    all_info = []

    rad2deg = 180.0 / np.pi
    earth_radius = 6378.137

    while epoch <= end_epoch:
        # print(epoch)
        year = epoch.year
        month = epoch.month
        date = epoch.day
        hour = epoch.hour
        minute = epoch.minute
        second = epoch.second

        now_time = f'{year}-{month}-{date} {hour}:{minute}:{second}'
        temp = change_df[change_df['time'] <= pd.to_datetime(now_time)].copy()
        tle1, tle2 = temp.iloc[-1]['first_line'], temp.iloc[-1]['second_line']

        satellite = Satrec.twoline2rv(tle1, tle2)
        jd, fr = jday(year, month, date, hour, minute, second)
        e, position, velocity = satellite.sgp4(jd, fr)

        # 추가된 부분
        ts = load.timescale()
        t = ts.utc(year, month, date, hour, minute, second)
        earth_satellite = EarthSatellite(tle1, tle2, '', ts)
        geocentric = earth_satellite.at(t)
        subpoint = wgs84.subpoint(geocentric)
        latitude = subpoint.latitude.degrees
        longitude = subpoint.longitude.degrees

        all_info.append([
            len(temp), tle1, tle2,
            epoch.year, epoch.month, epoch.day, epoch.hour, epoch.minute, epoch.second,
            *position, *velocity,
            (sum(p ** 2 for p in
                 position) ** 0.5) - earth_radius,
            (sum(v ** 2 for v in velocity) ** 0.5) / earth_radius,
            longitude,
            satellite.alta * earth_radius,
            satellite.altp * earth_radius,
            satellite.inclo * rad2deg, satellite.ecco,
            satellite.nodeo
        ])

        epoch += timedelta(minutes=1)

    info_df = pd.DataFrame(all_info)

    info_df.columns = ['change', 'tle1', 'tle2',
                       'year', 'month', 'day', 'hour', 'minute', 'second',
                       'x', 'y', 'z', 'vx', 'vy', 'vz',
                       'altitude', 'velocity', 'longitude',
                       'apogee', 'perigee',
                       'inclination', 'eccentricity', 'raan']

    return info_df

In [6]:
number_lst = [44343, 44349, 44350, 44351, 44353, 44358, 58464, 46267, 53611, 53019, 51961, 47775, 51969, 58722, 56361,
              56289, 56783, 48018, 43823, 45246, 29349, 37265, 42691, 42984, 55841]

to_do_lst = []

In [7]:
for number in number_lst:
    try:
        print(number)
        response = get_tle(number)
        tle_datas = response.text.split('\r\n')

        tle_first_lst = []
        tle_second_lst = []

        for i in range(0, len(tle_datas) - 2, 3):
            tle_first_lst.append(tle_datas[i + 1])
            tle_second_lst.append(tle_datas[i + 2])

        tle_df = pd.DataFrame({'first_line': tle_first_lst, 'second_line': tle_second_lst})
        tle_df.drop_duplicates(keep='first', inplace=True)
        tle_df.reset_index(inplace=True, drop=True)

        change_times = []
        for i in range(len(tle_df)):
            tle_1, tle_2 = tle_df['first_line'].iloc[i], tle_df['second_line'].iloc[i]
            satellite = Satrec.twoline2rv(tle_1, tle_2)
            jdsatepoch = satellite.jdsatepoch
            jdsatepochfrac = satellite.jdsatepochF
            epochdatetime = invjday(jdsatepoch + jdsatepochfrac)
            change_times.append(epochdatetime)

        change_df = pd.DataFrame(change_times)
        change_df.columns = ['year', 'month', 'day', 'hour', 'minute', 'second']
        change_df['first_line'] = tle_df['first_line']
        change_df['second_line'] = tle_df['second_line']
        change_df['time'] = pd.to_datetime(change_df[['year', 'month', 'day', 'hour', 'minute', 'second']])
        change_df['time'].drop_duplicates(keep='first', inplace=True)

        end_point = change_df.iloc[-1:].copy()
        end_point['day'] = end_point['day'] + 1
        end_point['hour'] = 0
        end_point['minute'] = 0
        end_point['second'] = 0
        end_point['time'] = pd.to_datetime(end_point[['year', 'month', 'day', 'hour', 'minute', 'second']])

        change_df = pd.concat([change_df, end_point], axis=0).copy()
        change_df.reset_index(inplace=True, drop=True)
        day_change_idx = list(change_df[['year', 'month', 'day']].drop_duplicates().index)

        seg_lst = []

        for i in range(len(day_change_idx) - 1):
            seg_lst.append(change_df.iloc[day_change_idx[i]:day_change_idx[i + 1] + 1].copy())

        results = Parallel(n_jobs=5)(delayed(get_info)(seg) for seg in tqdm(seg_lst))
        rslt = pd.concat(results)

        rslt.sort_values(by=['year', 'month', 'day', 'hour', 'minute', 'second']).reset_index(drop=True).to_csv(
            f'Database/{number}.csv')

    except:
        to_do_lst.append(number)

44343


100%|██████████| 536/536 [16:07<00:00,  1.80s/it]


44349


100%|██████████| 544/544 [16:02<00:00,  1.77s/it]


44350


...

100%|██████████| 539/539 [16:05<00:00,  1.79s/it]


42984


100%|██████████| 548/548 [16:09<00:00,  1.77s/it]


55841


100%|██████████| 485/485 [14:08<00:00,  1.75s/it]


In [8]:
print(to_do_lst)

[]


In [9]:
# name = 'longitude'
# df = rslt[['year', 'month', 'day', 'hour', 'minute', 'second', name]]
# 
# df_1 = df.copy()
# df_1.loc[:, 'datetime'] = pd.to_datetime(df_1[['year', 'month', 'day', 'hour', 'minute', 'second']])
# df_1.set_index('datetime', inplace=True)
# df_1.drop(columns=['year', 'month', 'day', 'hour', 'minute', 'second'], inplace=True)
# 
# hour_df = df_1[::60].copy()
# diff_df = np.log(hour_df[name]).diff().fillna(0)
# 
# draw_graph(diff_df, name.upper(), 'km',
#            pd.DataFrame(diff_df.iloc[[]].index.date).drop_duplicates().values.tolist(), "blue")